<h1> GYM LOCATION </h1>
<h3>first we download population data from Tunisian National Institut of Statistics </h3>
<h4> the link : <a href='http://www.ins.nat.tn/'>http://www.ins.nat.tn/ </a></h4>
<h3>after that we read the data </h3>   

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!pip install geopy 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bf3fbce9934b4b5eb2c79a5b6d99b278 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='**************************',
    ibm_auth_endpoint="**************************",
    config=Config(signature_version='********'),
    endpoint_url='**********************')

body = client_bf3fbce9934b4b5eb2c79a5b6d99b278.get_object(Bucket='***************',Key='*********')['Body']
body1 = client_bf3fbce9934b4b5eb2c79a5b6d99b278.get_object(Bucket='************',Key='***************')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
if not hasattr(body1, "__iter__"): body1.__iter__ = types.MethodType( __iter__, body1 )
    
    
CLIENT_ID = '***************' # your Foursquare ID
CLIENT_SECRET = '******************' # your Foursquare Secret
VERSION = '*******'
LIMIT = 30


df= pd.read_excel(body)
df.head()


,population,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,4 - 0 \nans,9 -5\n ans,14 - 10 \n ans,19 - 15 \n ans,29 - 20\n ans,39 - 30\n ans,49- 40 \n ans,59 - 50\nans,60 \n ans et plus
1,Carthage,6.54,6.45,5.72,6.67,17.62,14.09,13.98,13.19,15.74
2,La Medina,5.24,5.13,6.12,7.12,16.87,14.33,15.52,14.52,15.15
3,Bab Bhar,5.08,4.36,4.31,5.26,24.29,19.08,12.9,10.9,13.82
4,Bab Souika,5.73,5.31,5.6,6.29,17.47,15.17,14.56,14.15,15.72


<h3>we use this data to find wich neighbourhood have the highest % population with age between 15 to 50 (gym visitor) </h3>
<h2>but first , we must clean the data </h2>

In [3]:
#delete Unnamed row
df.iloc[0][0]='Neighborhood'
new_header = df.iloc[0]
df = df[1:]
for x in range (len(new_header)) :
    new_header[x]=new_header[x].replace('\n', "")
    new_header[x]=new_header[x].replace('ans', "year")


df.columns =new_header
df.head()

,Neighborhood,4 - 0 year,9 -5 year,14 - 10 year,19 - 15 year,29 - 20 year,39 - 30 year,49- 40 year,59 - 50year,60 year et plus
1,Carthage,6.54,6.45,5.72,6.67,17.62,14.09,13.98,13.19,15.74
2,La Medina,5.24,5.13,6.12,7.12,16.87,14.33,15.52,14.52,15.15
3,Bab Bhar,5.08,4.36,4.31,5.26,24.29,19.08,12.9,10.9,13.82
4,Bab Souika,5.73,5.31,5.6,6.29,17.47,15.17,14.56,14.15,15.72
5,Elomrane,7.61,6.59,6.43,6.76,16.35,15.44,15.24,12.67,12.91


<h3>Select our target by age</h3>

In [4]:
df1=df.iloc[: , [0, 4,5,6,7]]
df1.head()

,Neighborhood,19 - 15 year,29 - 20 year,39 - 30 year,49- 40 year
1,Carthage,6.67,17.62,14.09,13.98
2,La Medina,7.12,16.87,14.33,15.52
3,Bab Bhar,5.26,24.29,19.08,12.9
4,Bab Souika,6.29,17.47,15.17,14.56
5,Elomrane,6.76,16.35,15.44,15.24


In [5]:
s=[0]
for i in range (len(df1)):
    s.append(df1.iloc[i,1:5].sum())

df1['age_value_sum']=pd.DataFrame(s)
df1.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,19 - 15 year,29 - 20 year,39 - 30 year,49- 40 year,age_value_sum
1,Carthage,6.67,17.62,14.09,13.98,52.36
2,La Medina,7.12,16.87,14.33,15.52,53.84
3,Bab Bhar,5.26,24.29,19.08,12.9,61.53
4,Bab Souika,6.29,17.47,15.17,14.56,53.49
5,Elomrane,6.76,16.35,15.44,15.24,53.79


<h3>now we search the coordinate of neighobrhoods </h3>

In [6]:
import numpy as np
df1['latitude']=pd.Series([0] * 24)
df1['longitude']=pd.Series([0] * 24)
df1


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Neighborhood,19 - 15 year,29 - 20 year,39 - 30 year,49- 40 year,age_value_sum,latitude,longitude
1,Carthage,6.67,17.62,14.09,13.98,52.36,0,0
2,La Medina,7.12,16.87,14.33,15.52,53.84,0,0
3,Bab Bhar,5.26,24.29,19.08,12.9,61.53,0,0
4,Bab Souika,6.29,17.47,15.17,14.56,53.49,0,0
5,Elomrane,6.76,16.35,15.44,15.24,53.79,0,0
6,Elomrane Supérieur,6.54,17.76,16.82,13.68,54.80,0,0
7,Ettahrir,6.6,16.03,16.49,14.39,53.51,0,0
8,El Menzah,6.46,17.53,14.57,12.65,51.21,0,0
9,Cité Elkhadra,5.6,20.76,17.04,13.67,57.07,0,0
10,Le Bardo,6.26,17.03,15.73,14.58,53.60,0,0


In [7]:
from geopy.geocoders import Nominatim
for i in range (1,len(df1)-1):
    
    address = df1['Neighborhood'][i]+',Tunis,Tunisia'
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    
    if location:
        df1['latitude'][i] = location.latitude
        df1['longitude'][i] = location.longitude
        
    else:
        df1['latitude'][i] = 'None'
        df1['longitude'][i] = 'None'
        print('we cant find the location of :',address)
        
        
        

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipyk

we cant find the location of : Elomrane Supérieur,Tunis,Tunisia
we cant find the location of : Cité Elkhadra,Tunis,Tunisia
we cant find the location of : Elhrairia,Tunis,Tunisia
we cant find the location of : El Ourdia,Tunis,Tunisia
we cant find the location of : Djbel Djloud,Tunis,Tunisia


In [8]:
df1

,Neighborhood,19 - 15 year,29 - 20 year,39 - 30 year,49- 40 year,age_value_sum,latitude,longitude
1,Carthage,6.67,17.62,14.09,13.98,52.36,36,10
2,La Medina,7.12,16.87,14.33,15.52,53.84,36,10
3,Bab Bhar,5.26,24.29,19.08,12.9,61.53,36,10
4,Bab Souika,6.29,17.47,15.17,14.56,53.49,36,10
5,Elomrane,6.76,16.35,15.44,15.24,53.79,36,10
6,Elomrane Supérieur,6.54,17.76,16.82,13.68,54.80,None,None
7,Ettahrir,6.6,16.03,16.49,14.39,53.51,36.8271,10.1269
8,El Menzah,6.46,17.53,14.57,12.65,51.21,36.8356,10.1584
9,Cité Elkhadra,5.6,20.76,17.04,13.67,57.07,None,None
10,Le Bardo,6.26,17.03,15.73,14.58,53.60,36.8134,10.1322


<h3>we delete neighborhood who hasn't a location in this dataFrame</h3>
<h4 color ='red'>NB :if we use an other API or we collect more data may be we find the location of all neighborhood</h4>

In [9]:
df1 = df1.replace(to_replace='None', value=np.nan).dropna()
df1.drop([22,23],inplace=True)

In [10]:
df1.iloc[0, df1.columns.get_loc('latitude')] = 36.858154708278946
df1.iloc[0, df1.columns.get_loc('longitude')]=10.336781933624707
df1.iloc[1, df1.columns.get_loc('latitude')]=36.80761234097368
df1.iloc[1, df1.columns.get_loc('longitude')]=10.171900016252494
df1.iloc[2, df1.columns.get_loc('latitude')]=36.796856828491784
df1.iloc[2, df1.columns.get_loc('longitude')]=10.176148635332083
df1.iloc[3, df1.columns.get_loc('latitude')]=36.8044465832733
df1.iloc[3, df1.columns.get_loc('longitude')]=10.16555100837445
df1.iloc[4, df1.columns.get_loc('latitude')]=36.81616301530516
df1.iloc[4, df1.columns.get_loc('longitude')]=10.162096323163269
df1.head()

,Neighborhood,19 - 15 year,29 - 20 year,39 - 30 year,49- 40 year,age_value_sum,latitude,longitude
1,Carthage,6.67,17.62,14.09,13.98,52.36,36.858155,10.336782
2,La Medina,7.12,16.87,14.33,15.52,53.84,36.807612,10.171900
3,Bab Bhar,5.26,24.29,19.08,12.90,61.53,36.796857,10.176149
4,Bab Souika,6.29,17.47,15.17,14.56,53.49,36.804447,10.165551
5,Elomrane,6.76,16.35,15.44,15.24,53.79,36.816163,10.162096


In [11]:
df1=df1.sort_values(by=['age_value_sum'],ascending=False)
  



In [15]:
!pip install folium
import folium
from folium import plugins

m = folium.Map(location=[36.796857, 10.294356], zoom_start=12)

# Add the color for the chloropleth:
"""m.choropleth(
 geo_data=d,
 name='choropleth',
 data=df1,
 columns=['Neighborhood', 'age_value_sum'],
 key_on='properties.NAME_EN',
 fill_color='YlGn',
 fill_opacity=0.8,
 line_opacity=0.2,
 legend_name='population'
)
folium.LayerControl().add_to(m)"""


incidents = folium.plugins.MarkerCluster().add_to(m)
opc=0.8
i=0
clr='green'
for lat, lng, label in zip(df1.latitude, df1.longitude, df1.Neighborhood):
    i=i+1
    if i>6 :
        clr='red'
        opc=i*0.1-0.5
    else:
        opc=opc-0.05
    folium.CircleMarker(
        [lat, lng],
        radius=25,
        color=clr,
        popup=label,
        fill = True,
        fill_color=clr,
        fill_opacity=opc
    ).add_to(m)

m


In [17]:
latitude =36.807612
longitude =10.171900
search_query = 'gym'
radius = 10000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=HGIAJWFJ1SFIEKZV404RF5PJMAT4OCHUSFJPERF4TG1TNBEU&client_secret=HNVFNQTOLDZKALTVB3OUZINTTUII1VFJQCRMZGIUD3TIORDO&ll=36.807612,10.1719&v=20180604&query=gym&radius=10000&limit=30'

In [18]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dc7948566fc653db760469a'},
 'response': {'venues': [{'id': '54d10df4498e94d3f311fcd0',
    'name': 'California Gym',
    'location': {'lat': 36.80984090312947,
     'lng': 10.183319606741458,
     'labeledLatLngs': [{'label': 'display',
       'lat': 36.80984090312947,
       'lng': 10.183319606741458}],
     'distance': 1047,
     'cc': 'TN',
     'city': 'تونس',
     'state': 'Gouvernorat de Tunis',
     'country': 'تونس',
     'formattedAddress': ['تونس', 'تونس']},
    'categories': [{'id': '4bf58dd8d48988d176941735',
      'name': 'Gym',
      'pluralName': 'Gyms',
      'shortName': 'Gym',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1573360773',
    'hasPerk': False},
   {'id': '58beebf2abc6d6395bef1397',
    'name': 'Galaxy Gym',
    'location': {'address': 'Rue Hassinette',
     'lat': 36.822597849082825,
     'lng': 10.18665432929992

In [19]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,54d10df4498e94d3f311fcd0,NaN,TN,تونس,تونس,NaN,1047,"[تونس, تونس]","[{'label': 'display', 'lat': 36.80984090312947...",36.809841,10.183320,NaN,NaN,Gouvernorat de Tunis,California Gym,v-1573360773
1,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,58beebf2abc6d6395bef1397,Rue Hassinette,TN,تونس,تونس,NaN,2124,"[Rue Hassinette, تونس, تونس]","[{'label': 'display', 'lat': 36.82259784908282...",36.822598,10.186654,Montplaisir,NaN,Gouvernorat de Tunis,Galaxy Gym,v-1573360773
2,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,56d49a23cd105c886f8a9b1f,NaN,TN,NaN,تونس,NaN,1453,[تونس],"[{'label': 'display', 'lat': 36.80112262143136...",36.801123,10.186049,NaN,NaN,NaN,5ème Avenue GyM,v-1573360773
3,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,4defbdaf227170314bab2fcb,"32, rue Manoubi Bel Hassine",TN,أريانة‎,تونس,NaN,6261,"[32, rue Manoubi Bel Hassine, أريانة‎, تونس]","[{'label': 'display', 'lat': 36.86030457226683...",36.860305,10.147317,Ennaser II,NaN,Gouvernorat de l’ Ariana,California Gym,v-1573360773
4,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,594ab306e97dfb45c84fdd56,NaN,TN,NaN,تونس,NaN,5020,[تونس],"[{'label': 'display', 'lat': 36.85251, 'lng': ...",36.852510,10.166544,NaN,NaN,NaN,Gym Box,v-1573360773


In [20]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,California Gym,Gym,NaN,TN,تونس,تونس,NaN,1047,"[تونس, تونس]","[{'label': 'display', 'lat': 36.80984090312947...",36.809841,10.183320,NaN,NaN,Gouvernorat de Tunis,54d10df4498e94d3f311fcd0
1,Galaxy Gym,Gym / Fitness Center,Rue Hassinette,TN,تونس,تونس,NaN,2124,"[Rue Hassinette, تونس, تونس]","[{'label': 'display', 'lat': 36.82259784908282...",36.822598,10.186654,Montplaisir,NaN,Gouvernorat de Tunis,58beebf2abc6d6395bef1397
2,5ème Avenue GyM,Gym,NaN,TN,NaN,تونس,NaN,1453,[تونس],"[{'label': 'display', 'lat': 36.80112262143136...",36.801123,10.186049,NaN,NaN,NaN,56d49a23cd105c886f8a9b1f
3,California Gym,Gym,"32, rue Manoubi Bel Hassine",TN,أريانة‎,تونس,NaN,6261,"[32, rue Manoubi Bel Hassine, أريانة‎, تونس]","[{'label': 'display', 'lat': 36.86030457226683...",36.860305,10.147317,Ennaser II,NaN,Gouvernorat de l’ Ariana,4defbdaf227170314bab2fcb
4,Gym Box,Gym / Fitness Center,NaN,TN,NaN,تونس,NaN,5020,[تونس],"[{'label': 'display', 'lat': 36.85251, 'lng': ...",36.852510,10.166544,NaN,NaN,NaN,594ab306e97dfb45c84fdd56
5,Gold's Gym,Gym / Fitness Center,NaN,TN,NaN,تونس,NaN,4530,[تونس],"[{'label': 'display', 'lat': 36.83536743546226...",36.835367,10.134722,Cité Ibn Kaldoun,NaN,NaN,540f708d498ef1477a7e1583
6,Doctor Gym Soukra,Gym / Fitness Center,60 Avenue de l'Union Maghreb,TN,Sukrah,تونس,Borj Louzir,7284,"[60 Avenue de l'Union Maghreb (Borj Louzir), 2...","[{'label': 'display', 'lat': 36.86330252595908...",36.863303,10.214838,NaN,2073,Gouvernorat de Tunis,55361a4d498e5567e10d6679
7,CostaBrava Gym,General Entertainment,Rue Mohamed 5,TN,تونس,تونس,NaN,1776,"[Rue Mohamed 5, تونس, تونس]","[{'label': 'display', 'lat': 36.79893038451573...",36.798930,10.188626,NaN,NaN,Gouvernorat de Tunis,54db8305498ea092a7cb0e16
8,Body Form Gym,Gym,98 rue de la sebkha beb el jezira,TN,تونس,تونس,NaN,1972,"[98 rue de la sebkha beb el jezira, 1000 تونس,...","[{'label': 'display', 'lat': 36.79101186010570...",36.791012,10.179627,Beb el jezira,1000,Gouvernorat de Tunis,536a40a911d222f2d1414eb9
9,Feel Good Gym,Gym,20 Rue Hortensias,TN,Ariana,تونس,NaN,4900,"[20 Rue Hortensias, Ariana, تونس]","[{'label': 'display', 'lat': 36.85087538833389...",36.850875,10.182044,Nouvelle Ariana,NaN,Gouvernorat de l’ Ariana,4fa17317e4b0f72151906155


In [21]:
latitude =36.847999
longitude =10.294356
search_query = 'gym'
radius = 10000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered1 = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered1['categories'] = dataframe_filtered1.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered1.columns = [column.split('.')[-1] for column in dataframe_filtered1.columns]

dataframe_filtered1

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,California Gym Mall,Gym / Fitness Center,"5th floor, Tunisia Mall 2",TN,تونس,تونس,NaN,1209,"[5th floor, Tunisia Mall 2, تونس, تونس]","[{'label': 'display', 'lat': 36.846992, 'lng':...",36.846992,10.280834,les Jardins du Lac,NaN,Gouvernorat de Tunis,59fdae9ddec1d65351eaf714
1,California Gym Mall,Gym / Fitness Center,Mall 2,TN,Carthage,تونس,NaN,1221,"[Mall 2, 1053 Carthage, تونس]","[{'label': 'display', 'lat': 36.84709636674766...",36.847096,10.280690,NaN,1053,Gouvernorat de Tunis,5a0099d7bd4009409dbf6e20
2,Galaxy Gym,Gym / Fitness Center,Rue Hassinette,TN,تونس,تونس,NaN,10003,"[Rue Hassinette, تونس, تونس]","[{'label': 'display', 'lat': 36.82259784908282...",36.822598,10.186654,Montplaisir,NaN,Gouvernorat de Tunis,58beebf2abc6d6395bef1397
3,Doctor Gym Soukra,Gym / Fitness Center,60 Avenue de l'Union Maghreb,TN,Sukrah,تونس,Borj Louzir,7284,"[60 Avenue de l'Union Maghreb (Borj Louzir), 2...","[{'label': 'display', 'lat': 36.86330252595908...",36.863303,10.214838,NaN,2073,Gouvernorat de Tunis,55361a4d498e5567e10d6679
4,California Gym La Soukra,Gym / Fitness Center,NaN,TN,Ariana,تونس,NaN,3589,"[Ariana, تونس]","[{'label': 'display', 'lat': 36.87495, 'lng': ...",36.874950,10.272234,NaN,NaN,Gouvernorat de Tunis,5cdf25347b385f002c6d3aa8
5,California Gym,Gym,Rue du lac Biwa,TN,Les berges du lac,تونس,NaN,5031,"[Rue du lac Biwa, 1053 Les berges du lac, تونس]","[{'label': 'display', 'lat': 36.83355512969372...",36.833555,10.240842,NaN,1053,Gouvernorat de Tunis,4db9143b6a2334682da0a8c3
6,The Little Gym,Gymnastics Gym,Rue Echoghel,TN,La Goulette,تونس,NaN,2176,"[Rue Echoghel, La Goulette, تونس]","[{'label': 'display', 'lat': 36.830852, 'lng':...",36.830852,10.306086,NaN,NaN,Gouvernorat de Tunis,592056e02be425622acddcf4
7,Cimpress Gym,Gym / Fitness Center,NaN,TN,NaN,تونس,NaN,2198,[تونس],"[{'label': 'display', 'lat': 36.845016, 'lng':...",36.845016,10.269964,NaN,NaN,NaN,57ea2859498e05161962d184
8,Hot Milfs Gym,Athletics & Sports,NaN,TN,Tunis,تونس,NaN,2763,"[2045 Tunis, تونس]","[{'label': 'display', 'lat': 36.85764329342145...",36.857643,10.265769,NaN,2045,AIn zaghouan,51c1b44a8bbdcfaf6b652007
9,tejtonic gym,Gym,Gammarth,TN,Gammarth,تونس,NaN,2790,"[Gammarth, Gammarth, تونس]","[{'label': 'display', 'lat': 36.84793377007867...",36.847934,10.325679,NaN,NaN,Gouvernorat de l’ Ariana,5447f554498e578995fa7712


In [22]:

for lat, lng, label in zip(dataframe_filtered1.lat, dataframe_filtered1.lng, dataframe_filtered1.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(m)
for lat, lng, label in zip(dataframe_filtered1.lat, dataframe_filtered1.lng, dataframe_filtered1.categories):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(m)
# display map
m